# Проект: музыка больших городов

## Описание проекта

Сравнение Москвы и Петербурга окружено мифами: 
 - Москва — мегаполис, подчинённый жёсткому ритму рабочей недели;
 - Петербург — город своеобразной культуры, непохожий на Москву.

Некоторые мифы отражают действительность. Другие — пустые стереотипы. Бизнес должен отличать первые от вторых, чтобы принимать рациональные решения. Нам предлагается проверить гипотезы и сравнить поведение пользователей двух столиц на реальных данных Яндекс музыки.

## Цель исследования. Проверка гипотез.

1. Активность пользователей зависит от дня недели. Причём в Москве и Петербурге это проявляется по-разному.
2. Утром в понедельник в Москве преобладают одни жанры музыки, а в Петербурге — другие. Это верно и для вечера пятницы. 
3. Москва и Петербург предпочитают разные жанры музыки. В Москве чаще слушают поп-музыку, в Петербурге — русский рэп.

## Данные

Данные хранятся в файле yandex_music_project.csv

Описание колонок:
 - `userID` — идентификатор пользователя;
 - `Track` — название трека;
 - `artist` — имя исполнителя;
 - `genre` — название жанра;
 - `City` — город пользователя;
 - `time` — время начала прослушивания;
 - `Day` — день недели.

## Учебная задача

Проверка базовых знаний  Python и библиотеки pandas.

## Обзор данных

Составим первое представление о предоставленных данных:

In [21]:
# импорт библиотеки pandas
import pandas as pd

In [22]:
# чтение файла с данными и сохранение в df
df = pd.read_csv('datasets/yandex_music_project.csv')

Вывод первых строк таблицы

In [23]:
df.head()

,userID,Track,artist,genre,City,time,Day
0,FFB692EC,Kamigata To Boots,The Mass Missile,rock,Saint-Petersburg,20:28:33,Wednesday
1,55204538,Delayed Because of Accident,Andreas Rönnberg,rock,Moscow,14:07:09,Friday
2,20EC38,Funiculì funiculà,Mario Lanza,pop,Saint-Petersburg,20:58:07,Wednesday
3,A3DD03C9,Dragons in the Sunset,Fire + Ice,folk,Saint-Petersburg,08:37:09,Monday
4,E2DC1FAE,Soul People,Space Echo,dance,Moscow,08:34:34,Monday


Вывод общей информации о таблице

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65079 entries, 0 to 65078
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0     userID  65079 non-null  object
 1   Track     63848 non-null  object
 2   artist    57876 non-null  object
 3   genre     63881 non-null  object
 4     City    65079 non-null  object
 5   time      65079 non-null  object
 6   Day       65079 non-null  object
dtypes: object(7)
memory usage: 3.5+ MB


Таблица содержит семь столбцов. Тип во всех столбцах - `object`

Согласно документации к данным:
* `userID` — идентификатор пользователя;
* `Track` — название трека;  
* `artist` — имя исполнителя;
* `genre` — название жанра;
* `City` — город пользователя;
* `time` — время начала прослушивания;
* `Day` — день недели.

Количество значений в столбцах различается. Значит, в данных есть пропущенные значения.

**Выводы**

В каждой строке таблицы — данные о прослушанном треке. Часть колонок описывает саму композицию: название, исполнителя и жанр. Остальные данные рассказывают о пользователе: из какого он города, когда он слушал музыку. 

Предварительно можно утверждать, что, данных достаточно для проверки гипотез. Но встречаются пропуски в данных, а в названиях колонок — расхождения с хорошим стилем(строчные буквы сочетаются с прописными, встречаются пробелы).

Чтобы двигаться дальше, нужно устранить проблемы в данных.

## Предобработка данных

### Исправление заголовков

In [25]:
# вывод названий
df.columns

Index(['  userID', 'Track', 'artist', 'genre', '  City  ', 'time', 'Day'], dtype='object')

In [26]:
'''
Функция для обработки названий столбцов в DataFrame.
Функция принимает на вход список названий столбцов, удаляет пробелы до и после названия, 
заменяет пробелы между словами на нижние подчеркивания, и приводит все названия к нижнему регистру.

Параметры:
columns (pd.Index) - Индекс столбцов DataFrame.

Возвращает:
pd.Index - Индекс столбцов с обработанными названиями.
'''


def clean_column_names(columns: pd.Index) -> pd.Index:
    cleaned_columns = columns.str.strip().str.replace(' ', '_').str.lower()
    return cleaned_columns

In [27]:
# обработка названий колонок нашего df
df.columns = clean_column_names(df.columns)
display(df.columns)

df = df.rename(columns={'userid': 'user_id'})
df.columns

Index(['userid', 'track', 'artist', 'genre', 'city', 'time', 'day'], dtype='object')

Index(['user_id', 'track', 'artist', 'genre', 'city', 'time', 'day'], dtype='object')

### Обработка пропусков

In [28]:
# подстчет ропусков
df.isna().sum()

user_id       0
track      1231
artist     7203
genre      1198
city          0
time          0
day           0
dtype: int64

Пропуски в колонках `track` и `artist` не влияют на исследование, по этому  достаточно заменить их явными обозначеними. Пропуски в `genre` могут помешать сравнению музыкальных вкусов в Москве и Санкт-Петербурге.  Установить причину пропусков и восстановить данные возможности нет, придётся:
 - заполнить и эти пропуски явными обозначениями,
 - оценить, насколько они повредят расчётам.

In [31]:
# словарь, в котором ключи — названия столбцов, а значения — 'unknown'
values_to_replace = {'track': 'unknown', 'artist': 'unknown', 'genre': 'unknown'}

# замена пропущенных значений в указанных столбцах
df.fillna(value=values_to_replace, inplace=True)

# подстчет ропусков
df.isna().sum()


user_id    0
track      0
artist     0
genre      0
city       0
time       0
day        0
dtype: int64

### Обработка дубликатов

In [32]:
# подсчёт явных дубликатов
df.duplicated().sum()

np.int64(3826)

In [34]:
# удаление явных дубликатов (с удалением старых индексов и формированием новых)
df = df.drop_duplicates()

# проверка удаления явных дубликатов
df.duplicated().sum()

np.int64(0)

Так же нужна проверка неявных дубликатов в колонке `genre`, название одного и того же жанра может быть записано немного по-разному. Такие ошибки повлияют на результат исследования.

In [39]:
genres = df['genre'].sort_values().unique()
genres

array(['acid', 'acoustic', 'action', 'adult', 'africa', 'afrikaans',
       'alternative', 'alternativepunk', 'ambient', 'americana',
       'animated', 'anime', 'arabesk', 'arabic', 'arena',
       'argentinetango', 'art', 'audiobook', 'author', 'avantgarde',
       'axé', 'baile', 'balkan', 'beats', 'bigroom', 'black', 'bluegrass',
       'blues', 'bollywood', 'bossa', 'brazilian', 'breakbeat', 'breaks',
       'broadway', 'cantautori', 'cantopop', 'canzone', 'caribbean',
       'caucasian', 'celtic', 'chamber', 'chanson', 'children', 'chill',
       'chinese', 'choral', 'christian', 'christmas', 'classical',
       'classicmetal', 'club', 'colombian', 'comedy', 'conjazz',
       'contemporary', 'country', 'cuban', 'dance', 'dancehall',
       'dancepop', 'dark', 'death', 'deep', 'deutschrock', 'deutschspr',
       'dirty', 'disco', 'dnb', 'documentary', 'downbeat', 'downtempo',
       'drum', 'dub', 'dubstep', 'eastern', 'easy', 'electronic',
       'electropop', 'emo', 'entehno', '

Неявные дубликаты:
 - hop,
 - hip,
 - hip-hop

In [41]:
# замена неявных дубликатов
duplicates = ['hip', 'hop', 'hip-hop']
genre = 'hiphop'
df['genre'] = df['genre'].replace(duplicates, genre) 

# повторная проверка на неявные дубликаты
df['genre'].sort_values().unique() 

array(['acid', 'acoustic', 'action', 'adult', 'africa', 'afrikaans',
       'alternative', 'alternativepunk', 'ambient', 'americana',
       'animated', 'anime', 'arabesk', 'arabic', 'arena',
       'argentinetango', 'art', 'audiobook', 'author', 'avantgarde',
       'axé', 'baile', 'balkan', 'beats', 'bigroom', 'black', 'bluegrass',
       'blues', 'bollywood', 'bossa', 'brazilian', 'breakbeat', 'breaks',
       'broadway', 'cantautori', 'cantopop', 'canzone', 'caribbean',
       'caucasian', 'celtic', 'chamber', 'chanson', 'children', 'chill',
       'chinese', 'choral', 'christian', 'christmas', 'classical',
       'classicmetal', 'club', 'colombian', 'comedy', 'conjazz',
       'contemporary', 'country', 'cuban', 'dance', 'dancehall',
       'dancepop', 'dark', 'death', 'deep', 'deutschrock', 'deutschspr',
       'dirty', 'disco', 'dnb', 'documentary', 'downbeat', 'downtempo',
       'drum', 'dub', 'dubstep', 'eastern', 'easy', 'electronic',
       'electropop', 'emo', 'entehno', '

**Выводы по предобработке**

Обнаружино три проблемы в данных:
 - нарушения в стиле заголовков,
 - пропущенные значения,
 - дубликаты — явные и неявные.

Сделано:
 - Исправлены заголовки, чтобы упростить работу с таблицей.
 - Пропущенные значения заменены на 'unknown'. Ещё предстоит увидеть, не повредят ли исследованию пропуски в колонке genre.
 - Без дубликатов исследование станет более точным.

Теперь можно перейти к проверке гипотез.

## Проверка гипотез

### Сравнение поведения пользователей двух столиц

Первая гипотеза утверждает, что пользователи по-разному слушают музыку в Москве и Санкт-Петербурге. Нужно проверить это предположение по данным о трёх днях недели — понедельнике, среде и пятнице. Для этого:

* Разделим пользователей Москвы и Санкт-Петербурга
* Сравним, сколько треков послушала каждая группа пользователей в понедельник, среду и пятницу.

In [43]:
# подсчёт прослушиваний в каждом городе
df.groupby('city')['genre'].count()  

city
Moscow              42741
Saint-Petersburg    18512
Name: genre, dtype: int64

В Москве прослушиваний больше, чем в Петербурге. Из этого не следует, что московские пользователи чаще слушают музыку. Просто самих пользователей в Москве больше.

Cгруппируем данные по дню недели и подсчитаем прослушивания в понедельник, среду и пятницу. В данных есть информация только о прослушиваниях только за эти дни.

In [44]:
# подсчёт прослушиваний в каждый из трёх дней
df.groupby('day')['day'].count()

day
Friday       21840
Monday       21354
Wednesday    18059
Name: day, dtype: int64

В среднем пользователи из двух городов менее активны по средам. Но картина может измениться, если рассмотреть каждый город в отдельности.

Создадим функцию, которая посчитает прослушивания для заданного дня и города. Ей понадобятся два параметра:
 - день недели,
 - название города.

In [45]:
def number_tracks(day: str, city: str) -> int:
    """
    Функция подсчитывает количество уникальных пользователей, прослушивающих треки
    в заданный день и в заданном городе.

    Параметры:
    day (str): День недели для фильтрации (например, 'Monday').
    city (str): Название города для фильтрации (например, 'Moscow').

    Возвращает:
    int: Количество уникальных пользователей, прослушивающих треки в указанный день и в указанном городе.
    """
    # Фильтруем данные на основе дня и города и считаем уникальные user_id
    track_list_count = len(df[(df['day'] == day) & (df['city'] == city)]['user_id'])
    return track_list_count


Получим данные о прослушиваниях в каждом городе в каждый из дней.

In [54]:
# количество прослушиваний в Москве по понедельникам
number_tracks('Monday', 'Moscow')

15740

In [49]:
# количество прослушиваний в Санкт-Петербурге по понедельникам
number_tracks('Monday', 'Saint-Petersburg')

5614

In [50]:
# количество прослушиваний в Москве по средам
number_tracks('Wednesday', 'Moscow')

11056

In [51]:
# количество прослушиваний в Санкт-Петербурге по средам
number_tracks('Wednesday', 'Saint-Petersburg')

7003

In [52]:
# количество прослушиваний в Москве по пятницам
number_tracks('Friday', 'Moscow')

15945

In [53]:
# количество прослушиваний в Санкт-Петербурге по пятницам
number_tracks('Friday', 'Saint-Petersburg')

5895

Сведем полученные данные в таблицу

In [55]:
info = pd.DataFrame(
    data=[['Москва', number_tracks('Monday', 'Moscow'), 
           number_tracks('Wednesday', 'Moscow'), 
           number_tracks('Friday', 'Moscow')], ['Санкт-Петербург', 
           number_tracks('Monday', 'Saint-Petersburg'), 
           number_tracks('Wednesday', 'Saint-Petersburg'), 
           number_tracks('Friday', 'Saint-Petersburg')]], 
           columns=['city', 'monday', 'wednesday', 'friday'])
info

,city,monday,wednesday,friday
0,Москва,15740,11056,15945
1,Санкт-Петербург,5614,7003,5895


**Выводы**

Данные показывают разницу поведения пользователей:

- В Москве пик прослушиваний приходится на понедельник и пятницу, а в среду заметен спад.
- В Петербурге, наоборот, больше слушают музыку по средам. Активность в понедельник и пятницу здесь почти в равной мере уступает среде.

Значит, данные говорят в пользу первой гипотезы.

### Музыка в начале и в конце недели

Согласно второй гипотезе, утром в понедельник в Москве преобладают одни жанры, а в Петербурге — другие. Так же и вечером пятницы преобладают разные жанры — в зависимости от города.

Создадим две переменные с данными по Москве и с данными по Санкт-Петербургу

In [56]:
# получение таблицы moscow_general из тех строк таблицы df, 
# для которых значение в столбце 'city' равно 'Moscow'
moscow_general = df[df['city'] == 'Moscow']

# получение таблицы spb_general из тех строк таблицы df,
# для которых значение в столбце 'city' равно 'Saint-Petersburg'
spb_general = df[df['city'] == 'Saint-Petersburg']

Создадим функцию `genre_weekday()` с четырьмя параметрами:
* таблица (датафрейм) с данными,
* день недели,
* начальная временная метка в формате 'hh:mm', 
* последняя временная метка в формате 'hh:mm',

которая должна вернуть информацию о топ-10 жанров тех треков, которые прослушивали в указанный день, в промежутке между двумя отметками времени.

In [57]:
def genre_weekday(df: pd.DataFrame, day: str, time1: str, time2: str) -> pd.Series:
    """
    Функция возвращает 10 самых популярных жанров в указанный отрезок времени заданного дня.

    Параметры:
    df (pd.DataFrame): Датафрейм, содержащий данные с колонками 'day', 'time', и 'genre'.
    day (str): День недели, по которому требуется фильтрация (например, 'Monday').
    time1 (str): Начальное время в формате 'HH:MM' для фильтрации.
    time2 (str): Конечное время в формате 'HH:MM' для фильтрации.

    Возвращает:
    pd.Series: Сортированный по убыванию Series с 10 самыми популярными жанрами.
    """
    # Последовательная фильтрация данных
    genre_df = df[(df['day'] == day) & (df['time'] > time1) & (df['time'] < time2)]
    
    # Группировка по жанрам и подсчет количества строк
    genre_df_grouped = genre_df['genre'].value_counts()
    
    # Возврат 10 самых популярных жанров
    return genre_df_grouped.head(10)

Cравним результаты функции `genre_weekday()` для Москвы и Санкт-Петербурга в понедельник утром (с 7:00 до 11:00) и в пятницу вечером (с 17:00 до 23:00):

In [58]:
# вызов функции для утра понедельника в Москве
genre_weekday(moscow_general, 'Monday', '07:00', '11:00')

genre
pop            781
dance          549
electronic     480
rock           474
hiphop         286
ruspop         186
world          181
rusrap         175
alternative    164
unknown        161
Name: count, dtype: int64

In [59]:
# вызов функции для утра понедельника в Петербурге
genre_weekday(spb_general, 'Monday', '07:00', '11:00')

genre
pop            218
dance          182
rock           162
electronic     147
hiphop          80
ruspop          64
alternative     58
rusrap          55
jazz            44
classical       40
Name: count, dtype: int64

In [60]:
# вызов функции для вечера пятницы в Москве
genre_weekday(moscow_general, 'Friday', '17:00', '23:00')

genre
pop            713
rock           517
dance          495
electronic     482
hiphop         273
world          208
ruspop         170
classical      163
alternative    163
rusrap         142
Name: count, dtype: int64

In [61]:
# вызов функции для вечера пятницы в Петербурге
genre_weekday(spb_general, 'Friday', '17:00', '23:00')

genre
pop            256
electronic     216
rock           216
dance          210
hiphop          97
alternative     63
jazz            61
classical       60
rusrap          59
world           54
Name: count, dtype: int64

**Выводы**

Если сравнить топ-10 жанров в понедельник утром, можно сделать такие выводы:

1. В Москве и Петербурге слушают похожую музыку. Единственное отличие — в московский рейтинг вошёл жанр “world”, а в петербургский — джаз и классика.

2. В Москве пропущенных значений оказалось так много, что значение `'unknown'` заняло десятое место среди самых популярных жанров. Значит, пропущенные значения занимают существенную долю в данных и угрожают достоверности исследования.

Вечер пятницы не меняет эту картину. Некоторые жанры поднимаются немного выше, другие спускаются, но в целом топ-10 остаётся тем же самым.

Таким образом, вторая гипотеза подтвердилась лишь частично:
* Пользователи слушают похожую музыку в начале недели и в конце.
* Разница между Москвой и Петербургом не слишком выражена. В Москве чаще слушают русскую популярную музыку, в Петербурге — джаз.

Однако пропуски в данных ставят под сомнение этот результат. В Москве их так много, что рейтинг топ-10 мог бы выглядеть иначе, если бы не утерянные  данные о жанрах.

### Жанровые предпочтения в Москве и Петербурге

Гипотеза: Петербург — столица рэпа, музыку этого жанра там слушают чаще, чем в Москве.  А Москва — город контрастов, в котором, тем не менее, преобладает поп-музыка.

 - Сгруппируем таблицу `moscow_general` по жанру и посчитайте прослушивания треков каждого жанра;
 - Отсортируйте результат в порядке убывания;
 - Сохраним его в таблице `moscow_genres`.

In [63]:
moscow_genres = moscow_general.groupby('genre')['genre'].count()
moscow_genres = moscow_genres.sort_values(ascending=False)

# просмотр первых 10 строк moscow_genres
moscow_genres.head(10)


genre
pop            5892
dance          4435
rock           3965
electronic     3786
hiphop         2096
classical      1616
world          1432
alternative    1379
ruspop         1372
rusrap         1161
Name: genre, dtype: int64

Повторим те же действия для Петербурга

In [65]:
spb_genres = spb_general.groupby('genre')['genre'].count()
spb_genres = spb_genres.sort_values(ascending = False)

# просмотр первых 10 строк spb_genres
spb_genres.head(10)

genre
pop            2431
dance          1932
rock           1879
electronic     1736
hiphop          960
alternative     649
classical       646
rusrap          564
ruspop          538
world           515
Name: genre, dtype: int64

**Выводы**

Гипотеза частично подтвердилась:
* Поп-музыка — самый популярный жанр в Москве, как и предполагала гипотеза. Более того, в топ-10 жанров встречается близкий жанр — русская популярная музыка.
* Вопреки ожиданиям, рэп одинаково популярен в Москве и Петербурге. 

## Итоги исследования

Проверили три гипотезы и установили:

1. День недели по-разному влияет на активность пользователей в Москве и Петербурге. 

Первая гипотеза полностью подтвердилась.

2. Музыкальные предпочтения не сильно меняются в течение недели — будь то Москва или Петербург. Небольшие различия заметны в начале недели, по понедельникам:
* в Москве слушают музыку жанра “world”,
* в Петербурге — джаз и классику.

Таким образом, вторая гипотеза подтвердилась лишь отчасти. Этот результат мог оказаться иным, если бы не пропуски в данных.

3. Во вкусах пользователей Москвы и Петербурга больше общего чем различий. Вопреки ожиданиям, предпочтения жанров в Петербурге напоминают московские.

Третья гипотеза не подтвердилась. Если различия в предпочтениях и существуют, на основной массе пользователей они незаметны.